# Régression logistique - Analyse de sentiment des tweets

Vous allez apprendre la régression logistique. Concrètement, vous allez implémenter la régression logistique pour l'analyse de sentiment sur des tweets. À partir d'un tweet, vous déciderez s'il a un sentiment positif ou négatif. Plus précisément, vous allez :

* Apprendre à extraire des caractéristiques pour la régression logistique à partir d'un texte.
* Implémenter la régression logistique à partir de zéro.
* Appliquer la régression logistique à une tâche de traitement du langage naturel.
* Tester votre régression logistique.
* Effectuer une analyse des erreurs.

Nous utiliserons un ensemble de données de tweets. Espérons que vous obtiendrez une précision de plus de 99%. Exécutez la cellule ci-dessous pour charger les packages.

## Installation de librairies manquantes

In [ ]:
%pip install nltk

## Imports (librairies, fonctions et données)

In [ ]:
# Importe librairie nltk
import nltk
from os import getcwd

In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs # importe les fonctions d'aide à partir du fichier `utils.py.

### Préparation des données

* Les données `twitter_samples` contiennent des sous-ensembles de 5 000 tweets positifs, 5 000 tweets négatifs et l'ensemble complet de 10 000 tweets.

* Nous allons sélectionner uniquement les 5 000 tweets positifs et les 5 000 tweets négatifs.

In [ ]:
# Récupère les ensembles de tweets positifs and négatifs
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

**Instructions**: Vérifiez le nombre de tweets positifs et de tweets négatifs
<details>
<summary>Aide</summary>
Vous pouvez utiliser la fonction 
<a href="https://www.w3schools.com/python/ref_func_len.asp">`len(...)`</a>
 
</details>

In [ ]:
### Code ici


***Instructions***: Diviser les données en ensembles d'entraînement et de test, avec 20% des données dans l'ensemble de test et 80% dans l'ensemble d'entraînement.
<details>
<summary>Aide</summary>

* Pour sélectionner les X premiers éléments d'une liste L, vous pouvez utiliser `L[:X]`.
* Pour sélectionner les éléments restants (N-X) si la liste L est de longeur N, vous pouvez utiliser `L[X:]`.
</details>


In [ ]:
# Divise les données en deux ensembles, l'un pour l'entrainement et l'autre pour le test 
### Code ici - Début => Remplacez les valeurs None
test_pos = None
train_pos = None
test_neg = None
train_neg = None
### Code ici - Fin

# Combine les tweets positifs et négatifs des deux ensembles
train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [ ]:
# Vérifie la taille des ensembles d'entrainment et de test
print("len(train_x) = " + str(len(train_x)))
print("len(test_x) = " + str(len(test_x)))

In [ ]:
# Combine les étiquettes positives et négatives
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [ ]:
# Affiche la taille des ensembles d'entrainment et de test
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

* Créez le dictionnaire de fréquence en utilisant la fonction `build_freqs()` importée.
* Il est vivement recommandé d'ouvrir `utils.py` et de lire la fonction `build_freqs()` pour comprendre ce qu'elle fait.

* `build_freqs()` est une fonction qui compte combien de fois un mot dans le 'corpus' (l'ensemble complet des tweets) était associé à une étiquette positive '1' ou à une étiquette négative '0', puis construit le dictionnaire `freqs`, où chaque clé `key` est un tuple (mot, étiquette) et la valeur `value` est le compte de sa fréquence dans le corpus des tweets.

In [ ]:
# Crée le dictionnaire des férquences
freqs = build_freqs(train_x, train_y)

# Vérifie le résultat
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

**Instructions**: Affichez les 20 éléments du dictionnaire `freqs` avec les plus grandes valeurs (fréquences)
* Rappel: Chaque élément (`item`) du dictionnaire `freqs` a pour clé (`key`) une pair (mot, étiquette) et une valeur (`value`) correspondant à la fréquence. 


<details>
<summary>Aide</summary>

#### Exemple : Trier un dictionnaire par valeur dans l'ordre décroissant 
sorted_list = sorted(my_dict.items(), key = lambda x:x[1], reverse = True)
</details>



In [ ]:
### Code ici
sorted_freqs = None
print(sorted_freqs[:20])

### Process tweet
La fonction `process_tweet()` fournie tokenise le tweet en mots individuels, supprime les mots vides (stop words) et applique la racinisation (stemming).

In [ ]:
# test de la fonction
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

#### Expected output
```
This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
 
This is an example of the processes version: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']
```

# 1. Regression Logistique


### Sigmoide
$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

In [ ]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
  
    # calcule la sigmoide de z
    h = 1/(1+np.exp(-1*z))
    
    return h

### Regression Logistique: regression et sigmoide

La régression logistique prend une régression linéaire régulière et applique une fonction sigmoïde à la sortie de la régression linéaire.

Régression linéaire :
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Notez que les valeurs $\theta$ sont les "poids".

Régression logistique :
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$

### Partie 1.2 Fonction de coût et Gradient

La fonction de coût utilisée pour la régression logistique est la moyenne de la cross-entropie sur l'ensemble des exemples d'entraînement :

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ est le nombre d'exemples d'entraînement.
* $y^{(i)}$ est l'étiquette réelle du i-ème exemple d'entraînement.
* $h(z(\theta)^{(i)})$ est la prédiction du modèle pour le i-ème exemple d'entraînement.

La fonction de coût pour un seul exemple d'entraînement est
$$ -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

#### Mise à jour des poids

Pour mettre à jour le vecteur de poids $\theta$, vous appliquerez la descente de gradient pour améliorer itérativement les prédictions de votre modèle.

Le gradient de la fonction de coût $J$ par rapport à l'un des poids $\theta_j$ est donné par :

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$

* 'i' est l'indice à travers les 'm' exemples d'entraînement.
* 'j' est l'indice du poids $\theta_j$, donc $x_j$ est la caractéristique associée au poids $\theta_j$.

Pour mettre à jour le poids $\theta_j$, nous l'ajustons en soustrayant une fraction du gradient déterminée par $\alpha$ :

$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta)$$

* Le taux d'apprentissage $\alpha$ est une valeur que nous choisissons pour contrôler la taille d'une seule mise à jour.

## Instructions : Implémentez la fonction de descente de gradient
* Le nombre d'itérations `num_iters` est le nombre de fois que vous utiliserez l'ensemble complet d'entraînement.
* À chaque itération, vous calculerez la fonction de coût en utilisant tous les exemples d'entraînement (il y a `m` exemples d'entraînement) et pour toutes les caractéristiques.
* Au lieu de mettre à jour un seul poids $\theta_i$ à la fois, nous pouvons mettre à jour tous les poids dans le vecteur colonne :
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ a des dimensions (n+1, 1), où 'n' est le nombre de caractéristiques, et il y a un élément de plus pour le terme de biais $\theta_0$ (notez que la valeur de caractéristique correspondante $\mathbf{x_0}$ est 1).
* Les 'logits', 'z', sont calculés en multipliant la matrice de caractéristiques 'x' par le vecteur de poids 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ a des dimensions (m, n+1) 
    * $\mathbf{\theta}$ a des dimensions (n+1, 1)
    * $\mathbf{z}$ a des dimensions (m, 1)
* La prédiction 'h' est calculée en appliquant la sigmoïde à chaque élément de 'z' : $h(z) = sigmoid(z)$, et a des dimensions (m,1).
* La fonction de coût $J$ est calculée en prenant le produit scalaire des vecteurs 'y' et 'log(h)'.  Comme 'y' et 'h' sont tous deux des vecteurs colonnes (m,1), transposez le vecteur à gauche, de sorte que la multiplication matricielle d'un vecteur ligne avec un vecteur colonne effectue le produit scalaire.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* La mise à jour de $\theta$ est également vectorisée. Parce que les dimensions de $\mathbf{x}$ sont (m, n+1), et à la fois $\mathbf{h}$ et $\mathbf{y}$ sont (m, 1), nous devons transposer $\mathbf{x}$ et le placer à gauche afin de réaliser la multiplication matricielle, ce qui nous donne ensuite la réponse dont nous avons besoin de dimensions (n+1, 1) :
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

<details>
<summary>Aide</summary>
Pour le produit matriciel, vous pouvez utiliser la méthode 
<a href="https://numpy.org/doc/2.1/reference/generated/numpy.dot.html
">`np.dot`</a>
 
</details>


In [ ]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    # 'm', Nombre de ligne de la matrice x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        ### Code ici - Début
        # Calculez z, produit scalaire de x et theta (combinaiason linéaire)
        z = None
        
        # Calculez z la sigmoide de z
        h = None
        ### Code ici - Fin

        # Calcule la fonction de coût avec les m observations
        J = float(-1/m)*(np.transpose(y).dot(np.log(h))+np.transpose(1-y).dot(np.log(1-h)))
        
        # Met à jour les poids theta
        theta = theta-alpha*np.transpose(x).dot((h-y))/m
        
    
    J = float(J)
    return J, theta

In [ ]:
# Vérifiez la fonction
# Construire un cas de test synthétique
np.random.seed(1)
# X de taille 10 x 3
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y étiquettes de tailles 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)
# Intialisez les poids à zéros de taille 3x1
tmp_theta = np.zeros((3, 1))
# Initialisez le nombre de d'itétations à 10000 et le taux d'apprentissage à 1e-8
tmp_num_iters = 10000
tmp_alpha = 1e-8
# Appliquez la descente de gradient
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, tmp_theta, tmp_alpha, tmp_num_iters)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

## Partie 2 : Extraction des caractéristiques

* Étant donné une liste de tweets, extrayez les caractéristiques et stockez-les dans une matrice. Vous allez extraire deux caractéristiques.
    * La première caractéristique est le nombre de mots positifs dans un tweet.
    * La deuxième caractéristique est le nombre de mots négatifs dans un tweet.
* Ensuite, entraînez votre classificateur de régression logistique sur ces caractéristiques.
* Testez le classificateur sur un ensemble de validation.

### Instructions : Implémentez la fonction extract_features.
* Cette fonction prend en entrée un seul tweet.
* Traitez le tweet en utilisant la fonction `process_tweet()` importée et enregistrez la liste des mots du tweet.
* Parcourez chaque mot de la liste des mots traités
    * Pour chaque mot, vérifiez le dictionnaire `freqs` pour le décompte lorsque ce mot a une étiquette positive '1'. (Vérifiez la clé (mot, 1.0))
    * Faites de même pour le décompte lorsque le mot est associé à l'étiquette négative '0'. (Vérifiez la clé (mot, 0.0).)

<details>    
<summary>
    Aide
</summary>

* Assurez-vous de gérer les cas où la clé (mot, étiquette) n'est pas trouvée dans le dictionnaire.</li>
* Cherchez comment utiliser la méthode `.get()` d'un dictionnaire Python sur le web. Voici un <a href="https://www.programiz.com/python-programming/methods/dictionary/get" > exemple </a>.</li>
</details>    

In [ ]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenise, stem, et enlève les stopwords
    word_l = process_tweet(tweet)
    
    # features à 0
    x = np.zeros((1, 3)) 
    
    #biais fixé à 1
    x[0,0] = 1 
    
    ### Code ici - Début
    
    # Boucle sur chacun des mots
    for word in word_l:
        
        # Incrémente le score positif du mot avec sa fréquence dans les tweets positifs
        x[0,1] += None
        
        # Incrémente le score négatif du mot avec sa fréquence dans les tweets négatifs
        x[0,2] += None
        
    ### Code ici - Fin
    assert(x.shape == (1, 3))
    return x

In [ ]:
# Testez votre fonction

# test 1:
# test sur une donnée d'entrainement
tmp1 = extract_features(train_x[0], freqs)
print(train_x[0])
print(tmp1)

#### Expected output
```
[[1.00e+00 3.02e+03 6.10e+01]]
```

In [ ]:
# test 2:
# Vérifiez dans le cas où les mots ne sont pas dans le dictionnaire des frequences
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

#### Expected output
```
[[1. 0. 0.]]
```

## Partie 3: Entrainement du Modèle

In [ ]:
# collecter les features 'x' dans une matrice 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# Etiquettes correspondant à X
Y = train_y

# Applique la descente de gradient
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 7000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

# Partie 4: Testez votre regression logistique

Testez votre regression logistique sur de nouvelles données encore jamais vues par le modèle.

**Instructions** : Complétez la fonction `predict_tweet`
qui prédit si un tweet est positif ou négatif.

* Extraire les features du tweet
* Appliquez les poids du modèle appris pour calculer les combinaisons linéaires z
* Appliquez la fonction sigmoid aux combianaisons linéaires pour prédire la valeur entre 0 et 1.

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [ ]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### Code ici - Début
    
    # Extraire les features du tweet
    x = None
    
    # Calculez la prédiction
    y_pred = None
    
    ### Code ici - Fin
    
    return y_pred

In [ ]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, np.squeeze(predict_tweet(tweet, freqs, theta))))

In [ ]:
# Vérifiez avec vos propres tweets
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

## Vérifiez la performance sur l'ensemble de test

**Instructions:** Implementer `test_logistic_regression` 
* À partir des données de test et des poids de votre modèle entraîné, calculez la précision (accuracy) de votre modèle de régression logistique.
* Utilisez votre fonction `predict_tweet()` pour faire des prédictions sur chaque tweet de l'ensemble de test.
* Si la prédiction est > 0,5, définissez la classification du modèle `y_hat` à 1, sinon définissez la classification du modèle `y_hat` à 0.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Note</b></font>
</summary>
<p>
<ul>
    <li>Une prédiction est exacte lorsque `y_hat` est égal à `test_y`. On additionne toutes les instances où elles sont égales et on divise par `m`.</li>
    <li>np.asarray() convertit la `list` en un `numpy array`</li>
    <li>np.squeeze() transforme le `array` de dimension (m,1) en un array de dimension  (m,) </li>
</ul>
</p>

In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### Code ici - Début
    
    # Liste pour stocker les predictions
    y_hat = []
    
    for tweet in test_x:
        # Obtenir la prédiction pour le tweet
        y_pred = None
        
        if y_pred > 0.5:
            # Ajouter 1.0 à la fin de la liste y_hat
            None
        else:
            # Ajouter 0 à la fin de la liste y_hat
            None
    ### Code ici - Fin

    # Avec l'implémentation ci-dessus, y_hat est une `list`, et test_y est un `array` de dimension (m,1)
    # On convertit ici les deux en array uni-dimensionnels pour pouvoir les comparer avec l'opérateur '=='
    accuracy = float(1/len(test_y))*np.sum(np.squeeze(np.asarray(y_hat))==np.squeeze(test_y))

    
    
    return accuracy

In [ ]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

# Partie 5: Analyse des erreurs

Dans cette partie, vous verrez quelques tweets que votre modèle a mal classés.

**QUESTION**:

- Pourquoi pensez-vous que les mauvaises classifications se sont produites ? Plus précisément, quel type de tweets votre modèle classe-t-il incorrectement ?

In [ ]:
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (np.squeeze(y), np.squeeze(y_hat), ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

# Partie 6: Prediction avec votre propre tweet

In [ ]:
# Feel free to change the tweet below
my_tweet = 'Absolutely loved the movie! It was so entertaining and had a great cast.'

print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')